<center>
  <img src="https://www.inteli.edu.br/wp-content/uploads/2022/04/28103439/Logo-Container.png" />
</center>

# `[SI10-2024-P1]` - Prova Prática

Sistemas de Informação - Módulo 10 - Turma 4



**Nome**: *Escreva seu nome aqui.*

**ANTES DE PROSSEGUIR FAÇA UMA CÓPIA DESTE NOTEBOOK**

----

## Contexto

Uma plataforma online, que atua na venda de produtos, deseja otimizar sua interface para melhorar a experiência do usuário e aumentar as taxas de conversão. Recentemente, a empresa implementou um redesign de sua página web, e os registros dessa mudança estão detalhados nos dados coletados desde então. As variáveis registradas diariamente incluem o número de visitantes únicos, sessões, taxa de rejeição, páginas por sessão, principais origens do tráfego, tipos de dispositivos usados, eventos ocorridos, taxa de conversão e o tempo médio de sessão.

Com base nesses dados, a empresa considera realizar uma mudança similar para otimizar ainda mais sua plataforma. Antes de proceder, a equipe deseja empregar simulações de Monte Carlo para avaliar os possíveis impactos de um novo redesign na taxa de conversão, tempo médio de sessão e taxa de rejeição, utilizando análises estatísticas para prever tendências futuras e ajustar estratégias de maneira informada. Essa abordagem visa garantir que as decisões tomadas maximizem as taxas de conversões, fundamentais para o sucesso contínuo da plataforma.

## Importação do dataset e das bibliotecas

In [82]:
# Importação do dataset
# https://drive.google.com/file/d/1K_2ZDsrhheRDKoHN2ynfLCPGPFiS2CZr/view?usp=drive_link
!gdown 1K_2ZDsrhheRDKoHN2ynfLCPGPFiS2CZr

Downloading...
From: https://drive.google.com/uc?id=1K_2ZDsrhheRDKoHN2ynfLCPGPFiS2CZr
To: /content/ITL-SI10-2024-P1-dataset.csv
100% 7.03k/7.03k [00:00<00:00, 18.2MB/s]


In [83]:
# Importação das bibliotecas
import pandas as pd

# Transforma o dataset em DataFrame Pandas
df = pd.read_csv('ITL-SI10-2024-P1-dataset.csv')
print(df.head())

         Data  Visitantes Únicos  Sessões  Taxa de Rejeição (%)  \
0  2023-01-01              113.0    126.0             50.496714   
1  2023-01-02               90.0    157.0             49.438007   
2  2023-01-03               92.0    185.0             49.800231   
3  2023-01-04              130.0    206.0             50.251843   
4  2023-01-05              123.0    155.0             48.070931   

   Páginas por Sessão Principais Origens do Tráfego Dispositivos  Eventos  \
0            1.834201                        Direto      Desktop        8   
1            1.955760                        Social      Desktop        5   
2            2.285052                        Direto      Desktop        9   
3            2.325464                          Pago       Mobile        6   
4            2.267006                        Direto      Desktop        6   

   Taxa de Conversão (%)  Tempo Médio de Sessão (minutos)  
0               1.952083                         5.395516  
1             

## Questões

### Questão 1

**Análise Exploratória de Dados**: Faça a exploração dos dados, utilizando gráficos interativos. Identifique correlações entre as variáveis e apresente essas relações em uma matriz de correlação focada nas variáveis mais relevantes para a conversão de vendas.

In [84]:
df.head(2)

,Data,Visitantes Únicos,Sessões,Taxa de Rejeição (%),Páginas por Sessão,Principais Origens do Tráfego,Dispositivos,Eventos,Taxa de Conversão (%),Tempo Médio de Sessão (minutos)
0,2023-01-01,113.0,126.0,50.496714,1.834201,Direto,Desktop,8,1.952083,5.395516
1,2023-01-02,90.0,157.0,49.438007,1.955760,Social,Desktop,5,2.066180,4.680899


In [94]:
import plotly.express as px

def px_bar_value_sum(df, colunm):
  '''
  função para somar a quantidade de um valor único e colocar em um grafico de barras
  '''
  # obtendo a quantidade de cada valor
  value_counts_res = df[colunm].value_counts()

  fig = px.bar(value_counts_res,x= "count")
  fig.show()

In [95]:
px_bar_value_sum(df, "Dispositivos")
px_bar_value_sum(df, "Principais Origens do Tráfego")

In [56]:
def encode_values(df, feature):
  '''
  função para encodar os valores strings dentro do meu dataframe
  '''

  df[feature] = df[feature].astype('category')
  df[feature] = df[feature].cat.codes

In [59]:
# encodando valores
encode_values(df, "Dispositivos")
encode_values(df, "Principais Origens do Tráfego")

In [66]:
import plotly.graph_objects as go
import random
import numpy as np

# criando um df sem data para que possamos fazer a matriz de correlação
df_sem_data = df.loc[:, df.columns != "Data"]

# criando um df com a matriz de correlação
corr_matrix = df_sem_data.corr()

# plotando a matriz
fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = corr_matrix.columns,
        y = corr_matrix.index,
        z = np.array(corr_matrix),
        text=corr_matrix.values,
        texttemplate='%{text:.2f}'
    )
)
fig.show()

Podemos ver nesse gráfico algumas correlações interressantes:
- taxa de conversão e numero de Sessões
- taxa de conversão e numero de paginas por sessão
- taxa de conversão e tempo médio da sessão
- Paginas por sessão e tempo médio

### Questão 2

**Simulação de Monte Carlo**: Implemente uma simulação de Monte Carlo para avaliar os possíveis efeitos de um novo redesign na plataforma. Analise como diferentes variáveis podem impactar os resultados e discuta os cenários que esta simulação pode prever, ajudando a empresa a tomar decisões mais informadas sobre o redesign.

In [125]:
def calcular_prob(df, index, value):
  '''
  Função calcular_prob: Calcula a distribuição de probabilidade para uma coluna específica de um DataFrame.
  '''
  res = {}
  df_dict = df.set_index(index).to_dict()[value]
  total = sum(df_dict.values())
  for chave, valor in df_dict.items():
    res[chave] = valor/total


  if sum(res.values()) != 1:
    res["margem de erro"] = 1 - sum(res.values())

  return res

In [126]:
import numpy as np
from typing import List


class Camada:
    def __init__(self, probs: list, alternativas: list):
        self.probs = probs
        self.alternativas = list(map(str, alternativas))

        if sum(probs) != 1:
            raise Exception("A soma das probabilidades deve resultar em 1")

        if len(probs) != len(alternativas):
            raise Exception("Deve conter a mesma quantidade probabilidades e alternativas")


class GaltonBoard:
    def __init__(self):
        self.camadas = []

    def adicionar_camadas(self, camadas: List[Camada]):
        self.camadas.extend(camadas)

    def simular_montecarlo(self, num_iteracoes):
        if len(self.camadas) == 0:
            raise Exception("é necessário adicionar uma camada para fazer a simulação")

        todas_escolhas = {}
        for _ in range(num_iteracoes):
            caminho_percorrido = []
            for camada in self.camadas:
                ramificacao_escolida = np.random.choice(camada.alternativas, p= camada.probs)
                caminho_percorrido.append(ramificacao_escolida)

            caminho_percorrido_str = " - ".join(caminho_percorrido)

            if caminho_percorrido_str not in todas_escolhas.keys():
                todas_escolhas[caminho_percorrido_str] = 1

            else:
                todas_escolhas[caminho_percorrido_str] += 1

        return todas_escolhas


In [143]:
def fazer_somatoria_com_groupby(df, column):
  '''
  colocar valores em um range com steps do tamnaho da media menos desvio_padrao
  '''
  media = df[column].mean()
  desvio_padrao = df[column].std()

  res = df.groupby(pd.cut(df[column], int(media - desvio_padrao))).count()
  return df[column]

In [144]:
taxa_regeicao = fazer_somatoria_com_groupby(df, "Taxa de Rejeição (%)")
sessoes = fazer_somatoria_com_groupby(df, "Sessões")
visitas_unicas = fazer_somatoria_com_groupby(df, "Visitantes Únicos")
paginas_por_sessao = fazer_somatoria_com_groupby(df, "Páginas por Sessão")
tempo_medio_sessao = fazer_somatoria_com_groupby(df, "Tempo Médio de Sessão (minutos)")

In [145]:
taxa_regeicao

0                  NaN
1                  NaN
2                  NaN
3                  NaN
4                  NaN
                    ..
(44.18, 45.444]    NaN
(45.444, 46.707]   NaN
(46.707, 47.97]    NaN
(47.97, 49.233]    NaN
(49.233, 50.497]   NaN
Name: Taxa de Rejeição (%), Length: 84, dtype: float64

In [131]:
taxa_regeicao.columns

Index(['soma'], dtype='object')

### Questão 3

**Análise de Sensibilidade**: Realize uma análise de sensibilidade para examinar como alterações em diferentes variáveis de entrada afetam as previsões da taxa de conversão.

In [ ]:
# Escreva sua resposta nesta seção (Você pode criar quantas células de texto e de código forem necessárias)

## Entrega

Siga esses passos para entregar sua prova.

1. Crie um repositório no GitHub para essa entrega.

2. Submeta seu desenvolvimento neste repositório.

3. Garanta que o repositório é publicamente acessível (eliminatório).

4. No Google forms da prova, submeta a URL do repositório.